In [1]:
spark

Waiting for a Spark session to start...

Spark application_1540458187951_75086: Some(http://gw02.itversity.com:4041)

In [2]:
sc.getConf.getAll.filter(_._2.contains("/proxy/"))(0)._2

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1540458187951_75086

In [3]:
def getType(o: Any) = o.getClass.getCanonicalName

getType: (o: Any)String


In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getenv("HOME").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home + "/dataSets/spark-guide/data/retail-data/by-day/*.csv"

path = /user/kranthidr/dataSets/spark-guide/data/retail-data/by-day/*.csv


/user/kranthidr/dataSets/spark-guide/data/retail-data/by-day/*.csv

In [6]:
val staticDataFrame = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(path)

staticDataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [7]:
staticDataFrame.createOrReplaceTempView("retail_data")
val staticSchema = staticDataFrame.schema

staticSchema = StructType(StructField(InvoiceNo,StringType,true), StructField(StockCode,StringType,true), StructField(Description,StringType,true), StructField(Quantity,IntegerType,true), StructField(InvoiceDate,TimestampType,true), StructField(UnitPrice,DoubleType,true), StructField(CustomerID,DoubleType,true), StructField(Country,StringType,true))


StructType(StructField(InvoiceNo,StringType,true), StructField(StockCode,StringType,true), StructField(Description,StringType,true), StructField(Quantity,IntegerType,true), StructField(InvoiceDate,TimestampType,true), StructField(UnitPrice,DoubleType,true), StructField(CustomerID,DoubleType,true), StructField(Country,StringType,true))

In [8]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [9]:
staticDataFrame.show(truncate=false, numRows=5)

+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                    |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|580538   |23084    |RABBIT NIGHT LIGHT             |48      |2011-12-05 08:38:00|1.79     |14075.0   |United Kingdom|
|580538   |23077    |DOUGHNUT LIP GLOSS             |20      |2011-12-05 08:38:00|1.25     |14075.0   |United Kingdom|
|580538   |22906    |12 MESSAGE CARDS WITH ENVELOPES|24      |2011-12-05 08:38:00|1.65     |14075.0   |United Kingdom|
|580538   |21914    |BLUE HARMONICA IN BOX          |24      |2011-12-05 08:38:00|1.25     |14075.0   |United Kingdom|
|580538   |22467    |GUMBALL COAT RACK              |6       |2011-12-05 08:38:00|2.55     |14075.0   |United Kingdom|
+---------+---------+---------------------------

In [10]:
sc.version

2.3.0.2.6.5.0-292

In [11]:
import org.apache.spark.sql.functions.{window, column, desc, col}

val t1 = staticDataFrame
  .selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")
  .groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day"))
  .sum("total_cost")

t1 = [CustomerId: double, window: struct<start: timestamp, end: timestamp> ... 1 more field]


[CustomerId: double, window: struct<start: timestamp, end: timestamp> ... 1 more field]

In [12]:
 t1.show(numRows=5, truncate=false)

+----------+------------------------------------------+------------------+
|CustomerId|window                                    |sum(total_cost)   |
+----------+------------------------------------------+------------------+
|14799.0   |[2011-10-18 20:00:00, 2011-10-19 20:00:00]|157.7             |
|16210.0   |[2011-09-01 20:00:00, 2011-09-02 20:00:00]|344.88            |
|17790.0   |[2010-12-12 19:00:00, 2010-12-13 19:00:00]|154.8             |
|14541.0   |[2011-10-11 20:00:00, 2011-10-12 20:00:00]|146.0             |
|12682.0   |[2011-07-03 20:00:00, 2011-07-04 20:00:00]|474.36000000000007|
+----------+------------------------------------------+------------------+
only showing top 5 rows



In [13]:
staticDataFrame
  .selectExpr("CustomerId", "UnitPrice", "Quantity", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")
  .where("CustomerId == 16057.0").where(col("InvoiceDate") >= "2011-12-04 19:00:00")
.where(col("InvoiceDate") <= "2011-12-05 19:00:00").show()

+----------+---------+--------+----------+-------------------+
|CustomerId|UnitPrice|Quantity|total_cost|        InvoiceDate|
+----------+---------+--------+----------+-------------------+
|   16057.0|    12.75|      -1|    -12.75|2011-12-05 16:36:00|
|   16057.0|     9.95|      -2|     -19.9|2011-12-05 16:36:00|
|   16057.0|     4.95|      -1|     -4.95|2011-12-05 16:36:00|
+----------+---------+--------+----------+-------------------+



In [14]:
t1.take(1)

15311.0,"[2011-04-03 20:00:00.0,2011-04-04 20:00:00.0]",1688.6500000000008


In [15]:
t1.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[CustomerId#16, window#128], functions=[sum(total_cost#60)])
+- Exchange hashpartitioning(CustomerId#16, window#128, 200)
   +- *(1) HashAggregate(keys=[CustomerId#16, window#128], functions=[partial_sum(total_cost#60)])
      +- *(1) Project [named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) as double) = (cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) THEN (CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) + 1) ELSE CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) END + 0) - 1) * 86400000000) + 0), LongType, TimestampType), end, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongTy

In [16]:
t1.printSchema()

root
 |-- CustomerId: double (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- sum(total_cost): double (nullable = true)



In [17]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [18]:
val streamingDataFrame = spark.readStream
    .schema(staticSchema)
    .option("maxFilesPerTrigger", 1)
    .format("csv")
    .option("header", "true")
    .load(path)

streamingDataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [19]:
streamingDataFrame.explain() //# note the Error

== Physical Plan ==
StreamingRelation FileSource[/user/kranthidr/dataSets/spark-guide/data/retail-data/by-day/*.csv], [InvoiceNo#129, StockCode#130, Description#131, Quantity#132, InvoiceDate#133, UnitPrice#134, CustomerID#135, Country#136]


In [20]:
val purchaseByCustomerPerHour = streamingDataFrame
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")
  .groupBy(
    col("CustomerId"), window($"InvoiceDate", "1 day"))
  .sum("total_cost")

purchaseByCustomerPerHour = [CustomerId: double, window: struct<start: timestamp, end: timestamp> ... 1 more field]


[CustomerId: double, window: struct<start: timestamp, end: timestamp> ... 1 more field]

In [21]:
purchaseByCustomerPerHour.writeStream
    .format("memory")
    .queryName("customer_purchases")
    .outputMode("complete")
    .start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@1df42c66

In [22]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)
  .show(20)

+----------+------+---------------+
|CustomerId|window|sum(total_cost)|
+----------+------+---------------+
+----------+------+---------------+



In [25]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  """).orderBy(desc("window"), desc("sum(total_cost)"))
  .show(10, truncate=false)

+----------+------------------------------------------+------------------+
|CustomerId|window                                    |sum(total_cost)   |
+----------+------------------------------------------+------------------+
|null      |[2010-11-30 19:00:00, 2010-12-01 19:00:00]|12584.299999999988|
|13777.0   |[2010-11-30 19:00:00, 2010-12-01 19:00:00]|6585.16           |
|16029.0   |[2010-11-30 19:00:00, 2010-12-01 19:00:00]|3702.12           |
|16210.0   |[2010-11-30 19:00:00, 2010-12-01 19:00:00]|2474.7399999999993|
|12433.0   |[2010-11-30 19:00:00, 2010-12-01 19:00:00]|1919.1400000000008|
|17511.0   |[2010-11-30 19:00:00, 2010-12-01 19:00:00]|1825.74           |
|17850.0   |[2010-11-30 19:00:00, 2010-12-01 19:00:00]|1499.339999999999 |
|13408.0   |[2010-11-30 19:00:00, 2010-12-01 19:00:00]|1024.6800000000003|
|15485.0   |[2010-11-30 19:00:00, 2010-12-01 19:00:00]|950.09            |
|12583.0   |[2010-11-30 19:00:00, 2010-12-01 19:00:00]|855.86            |
+----------+-------------

In [24]:
// # purchaseByCustomerPerHour.writeStream
// #     .format("console")
// #     .queryName("customer_purchases_2")
// #     .outputMode("complete")
// #     .start()

//#check on Console

Name: Syntax Error.
Message: 
StackTrace: 

In [26]:
//streamingDataFrame.isStreaming

true